In [3]:
!pip install -qU  langchain-core
!pip install -qU langchain-community
!pip install -qU  crewai[tools] 
!pip install -qU langchain-groq

In [1]:
pip install -qU ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama


In [2]:
df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,MI,FT,Data Analyst,1125000,INR,14329,IN,100,IN,L
1,2022,EN,FT,Data Scientist,130000,USD,130000,US,0,US,M
2,2023,MI,FT,Data Analyst,150000,USD,150000,US,0,US,M
3,2023,MI,FT,Data Analyst,100000,USD,100000,US,0,US,M
4,2023,SE,FT,Machine Learning Engineer,261500,USD,261500,US,0,US,L


In [3]:
# connection = sqlite3.connect("salaries.db")
# df.to_sql(name="salaries", con=connection)

In [4]:
llm = Ollama(
    model="phi3"
)

In [5]:
db = SQLDatabase.from_uri("sqlite:///salaries.db")

In [6]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

Using Tool: list_tables


'salaries'

In [7]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("salaries"))

Using Tool: tables_schema

CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/


In [8]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")

Using Tool: execute_sql


"[(0, 2022, 'MI', 'FT', 'Data Analyst', 1125000, 'INR', 14329, 'IN', 100, 'IN', 'L'), (1, 2022, 'EN', 'FT', 'Data Scientist', 130000, 'USD', 130000, 'US', 0, 'US', 'M'), (2, 2023, 'MI', 'FT', 'Data Analyst', 150000, 'USD', 150000, 'US', 0, 'US', 'M'), (3, 2023, 'MI', 'FT', 'Data Analyst', 100000, 'USD', 100000, 'US', 0, 'US', 'M'), (4, 2023, 'SE', 'FT', 'Machine Learning Engineer', 261500, 'USD', 261500, 'US', 0, 'US', 'L')]"

In [9]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'\nSELECT * FROM salaries WHERE salary > 10000 LIMIT 5;\n\nSince none of the specified common mistakes were identified in the given query, it is already correctly written for its intended purpose. Therefore, we reproduce the original query as provided.\n\nOutput SQL Query: \n\nSELECT * FROM salaries WHERE salary > 10000 LIMIT 5;'

In [10]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [11]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [12]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

In [13]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [14]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [15]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [16]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=2,
    memory=False,
    output_log_file="crew.log",
)

In [17]:

inputs = {
    "query": "Average salary for Data Analyst role."
}

result = crew.kickoff(inputs=inputs)

 [2024-07-15 08:56:50][DEBUG]: == Working Agent: Senior Database Developer
 [2024-07-15 08:56:50][INFO]: == Starting Task: Extract data that is required for the query Average salary for Data Analyst role..
 

salaries

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/

 


SELECT AVG(salary) as AverageSalary FROM salaries WHERE job_title = 'Data Analyst';

There is no need to modify this q

2024-07-15 09:07:52,212 - 6844 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "c:\Users\Anoop Maurya\Documents\GitHub\SQL-Agent-using-CrewAI-and-Groq\.venv\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Anoop Maurya\Documents\GitHub\SQL-Agent-using-CrewAI-and-Groq\.venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Anoop Maurya\Documents\GitHub\SQL-Agent-using-CrewAI-and-Groq\.venv\Lib\site-packages\urllib3\connection.py", line 464, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Anoop Maurya\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Users\Anoop Maurya\Ap

KeyboardInterrupt: 

In [ ]:
print(result)

### Executive Summary
The analysis of the provided data reveals that the average salary for a Data Analyst role is $103,925.46. This figure is based on a single data point, indicating that Data Analysts are well-compensated professionals. The high average salary may be due to the high demand for data analysis skills and the importance of data-driven decision-making in many industries. This analysis provides a clear and concise answer to the question of what the average salary is for a Data Analyst role, based on the provided data.


In [ ]:
inputs = {
    "query": "How is the `Machine Learning Engineer` salary in USD is affected by remote positions"
}

result = crew.kickoff(inputs=inputs)

 [2024-07-15 08:32:40][DEBUG]: == Working Agent: Senior Database Developer
 [2024-07-15 08:32:40][INFO]: == Starting Task: Extract data that is required for the query How is the `Machine Learning Engineer` salary in USD is affected by remote positions.
 

salaries

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/

 

The original query looks good, but I'll reproduce it for completeness:



In [ ]:
print(result)

'**Executive Summary: Impact of Remote Positions on Machine Learning Engineer Salaries**\n============================================================\n\n### Key Findings\n\n* Machine Learning Engineers with 0% remote work have an average salary of $163,587.78.\n* Those with 50% remote work have an average salary of $125,263.43, 23.4% lower than those with no remote work.\n* Those with 100% remote work have an average salary of $160,192.16, 2.1% lower than those with no remote work.\n* Remote positions have a negative impact on Machine Learning Engineer salaries, with a more significant impact for those with 50% remote work compared to those with 100% remote work.'

In [ ]:
inputs = {
    "query": "How is the salaray in USD based on employment type and experience level? "
}

result = crew.kickoff(inputs=inputs)

 [2024-07-15 08:29:06][DEBUG]: == Working Agent: Senior Database Developer
 [2024-07-15 08:29:06][INFO]: == Starting Task: Extract data that is required for the query How is the salaray in USD based on employment type and experience level? .
 

salaries

 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TEXT, 
	salary_in_usd INTEGER, 
	employee_residence TEXT, 
	remote_ratio INTEGER, 
	company_location TEXT, 
	company_size TEXT
)

/*
3 rows from salaries table:
index	work_year	experience_level	employment_type	job_title	salary	salary_currency	salary_in_usd	employee_residence	remote_ratio	company_location	company_size
0	2022	MI	FT	Data Analyst	1125000	INR	14329	IN	100	IN	L
1	2022	EN	FT	Data Scientist	130000	USD	130000	US	0	US	M
2	2023	MI	FT	Data Analyst	150000	USD	150000	US	0	US	M
*/

 

SELECT employment_type, experience_level, AVG(salary_in_usd) AS avg_salary_usd FROM

In [ ]:
print(result)

**Executive Summary**

The analysis of the salary dataset reveals that experience level has a significant impact on salary, with Executive-level employees earning significantly more than Entry-level employees. Freelancers (FT) tend to have higher average salaries than other employment types across all experience levels.

**Key Findings:**

* Executive-level employees have the highest average salaries, with Freelancers (FT) having the highest average salary at $185,556.49.
* Senior-level employees have the second-highest average salaries, with Freelancers (FT) again having the highest average salary at $146,191.89.
* Freelancers (FT) tend to have higher average salaries across all experience levels, followed by Contractors (CT) and Full-time (FL) employees.
* Part-time (PT) employees have the lowest average salaries across all experience levels.
